In [32]:
# import the GIS class in gis module
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from time import strftime
import os, requests, zipfile
import pandas as pd
import geopandas as gpd

## Download data


In [18]:
feature_layer_id = "062eaa4b57024dffa9a77822986a86a7"
# download_url = f"https://www.pasda.psu.edu/download/dep/StorageTankLocations_Active{strftime('%Y')}_{strftime('%m')}.zip"
download_url = f"https://www.pasda.psu.edu/download/dep/StorageTankLocations_Active2024_05.zip"
download_path = os.path.abspath("./assets")

# Filename from the URL
filename = download_url.split('/')[-1]

# Full path to save the file
file_path = os.path.join(download_path, filename)

# Download the file
response = requests.get(download_url)

# Check if the request was successful
if response.status_code == 200:
    # Write content to the file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print(f"File downloaded successfully to: {file_path}")
else:
    print("Failed to download the file.")

File downloaded successfully to: d:\Eric-Personal\python\docs\Geoprocessing\assets\StorageTankLocations_Active2024_05.zip


## unzip downloaded data


In [33]:
# Extract the downloaded ZIP file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(download_path)

# Path to the extracted CSV file
shp_file_path = os.path.join(download_path, "StorageTankLocations_Active2024_05.shp")

# Read the downloaded CSV data
downloaded_data = gpd.read_file(shp_file_path)
downloaded_data.head()

,FACILITY_I,FACILITY_N,FACILITY_A,FACILITY_1,FACILITY_C,FACILITY_S,FACILITY_Z,COUNTY_ID,FACILITY_2,MUNICIPALI,...,TANK_OWN_3,TANK_OWN_4,TANK_OWN_5,TANK_OWN_6,PRIMARY_FA,SITE_ID,LATITUDE,LONGITUDE,TANK_INFOR,geometry
0,56-39595,CHEMSTREAM,NORTH STAR IND PARK COMMERCE DR,None,STOYSTOWN,PA,15563,2463,Somerset,2130,...,None,HOMER CITY,PA,15748-1422,695172,691484,40.137605,-79.025093,http://cedatareporting.pa.gov/Reportserver/Pag...,POINT (-8797033.113 4885958.858)
1,35-52629,THROOP DIVERSION CHAMBER,862 ENTERPRISE ST,None,DICKSON CITY,PA,18519,2442,Lackawanna,1403,...,None,OLYPHANT,PA,18447-0280,695938,645882,41.461955,-75.616175,http://cedatareporting.pa.gov/Reportserver/Pag...,POINT (-8417554.097 5080720.569)
2,11-39625,MARTIN GENERAL STORE 230,4057 ADMIRAL PEARY HWY,None,EBENSBURG,PA,15931-3919,2418,Cambria,570,...,None,BELLWOOD,PA,16617-1922,696735,693483,40.460940,-78.769425,http://cedatareporting.pa.gov/Reportserver/Pag...,POINT (-8768572.281 4933152.658)
3,15-39634,FREEDOM VILLAGE AT BRANDYWINE,15 FREEDOM BLVD,None,WEST BRANDYWINE,PA,19320-1549,2422,Chester,753,...,None,WEST BRANDYWINE,PA,19320-1549,697311,694230,40.013371,-75.790556,http://cedatareporting.pa.gov/Reportserver/Pag...,POINT (-8436966.101 4867885.507)
4,45-39639,ROCK HILL CONCRETE,UPPER CHERRY VALLEY ROAD,RD4 BOX 140,STROUDSBURG,PA,18360,2452,Monroe,1781,...,None,DOWNERS GROVE,IL,60515-5560,697481,694449,40.927418,-75.271720,http://cedatareporting.pa.gov/Reportserver/Pag...,POINT (-8379209.542 5001641.723)


## Log into ArcGIS Online


In [27]:
gis = GIS("pro")
print("Logged in as " + str(gis.properties.user.username))

Logged in as yxy180050@utdallas.edu_UTDEPPS


In [36]:
# Get the existing feature layer data

data_item = gis.content.get(itemid=feature_layer_id)
feature_layer_collection = FeatureLayerCollection.fromitem(data_item)

In [39]:
feature_layer_collection.layers[0].properties

{
  "currentVersion": 11.2,
  "id": 0,
  "name": "StorageTankLocations_Active2024_05",
  "type": "Feature Layer",
  "serviceItemId": "062eaa4b57024dffa9a77822986a86a7",
  "cacheMaxAge": 30,
  "displayField": "",
  "description": "",
  "copyrightText": "",
  "defaultVisibility": true,
  "editingInfo": {
    "lastEditDate": 1717537000401,
    "schemaLastEditDate": 1717537000401,
    "dataLastEditDate": 1717537000401
  },
  "relationships": [],
  "isDataVersioned": false,
  "hasContingentValuesDefinition": false,
  "supportsAppend": true,
  "supportsCalculate": true,
  "supportsASyncCalculate": true,
  "supportsTruncate": true,
  "supportsAttachmentsByUploadId": true,
  "supportsAttachmentsResizing": true,
  "supportsRollbackOnFailureParameter": true,
  "supportsStatistics": true,
  "supportsExceedsLimitStatistics": true,
  "supportsAdvancedQueries": true,
  "supportsValidateSql": true,
  "supportsCoordinatesQuantization": true,
  "supportsLayerOverrides": true,
  "supportsTilesAndBasicQu

## Overwrite features


In [43]:
existing_data = feature_layer_collection.query("*")

In [29]:
# Check for differences
if downloaded_data.equals(existing_data):
    print("No updates needed. Data is identical.")
else:
    # Append new data to the existing feature layer
    feature_layer_collection.manager.append(file_path)
    print("Data appended and feature layer updated successfully.")

{'success': True}

In [45]:
# Clean up downloaded files
os.remove(file_path)
os.remove(shp_file_path)